In [ ]:
#require "pkp"

In [ ]:
open Owl
open Gp

## The 3 cards problem

Simple simulation for the "3 cards" problem:

In [ ]:
type side =
  | Red
  | Blue

In [ ]:
let deck = [| Red, Red; Blue, Blue; Red, Blue |]

In [ ]:
let random_flip (s1, s2) = if Random.bool () then s1, s2 else s2, s1

A function that describes the random draw + random flipping:

In [ ]:
let draw () = random_flip deck.(Random.int 3)

In [ ]:
draw ()

Simulate many trials:

In [ ]:
let many_trials = List.init 100000 (fun _ -> draw ())

To estimate the probability that the second side is Red **given that the first side is known to be Red**, we can simply select all draws in which the first side was Red, and count the fraction of those for which the second side is Red, too:  

In [ ]:
let s1_red = List.filter (fun (s1, _) -> s1 = Red) many_trials

In [ ]:
let s1_and_s2_red = List.filter (fun (_, s2) -> s2 = Red) s1_red

In [ ]:
float (List.length s1_and_s2_red) /. float (List.length s1_red)

Theoretically, this probability is 2/3.

## Combining evidence with expectations

In [ ]:
let gaussian = Owl_stats.gaussian_pdf

In [ ]:
let density_props =
  [ barebone
  ; borders [ `bottom ]
  ; xtics (`regular [ -10.; 1. ])
  ; xlabel "x"
  ; set "key at screen 0.95, screen 0.9 top right"
  ; margins [ `right 0.8 ]
  ]

In [ ]:
let prod f g = fun x -> f x *. g x

In [ ]:
let _ =
  let fig (module P : Plot) =
    P.plot (F (gaussian ~mu:0. ~sigma:1., Mat.linspace (-5.) 5. 300)) density_props
  in
  Juplot.draw ~size:(600, 300) fig

In [ ]:
let _ =
  let xs = Mat.linspace (-5.) 5. 300 in
  let normalise v = Mat.(v /$ max' v) in
  let prior = gaussian ~mu:0. ~sigma:2. in
  let likelihood = gaussian ~mu:1. ~sigma:0.5 in
  let posterior = prod prior likelihood in
  let p f = L [ xs; normalise (Mat.map f xs) ] in
  let fig (module P : Plot) =
    P.plots
      [ item (p prior) ~style:"l lc 8 lw 2" ~legend:"prior"
      ; item (p likelihood) ~style:"l lc 3 lw 2" ~legend:"likelihood"
      ; item (p posterior) ~style:"l lc 7 lw 2" ~legend:"posterior"
      ]
      density_props
  in
  Juplot.draw ~size:(600, 300) fig

In [ ]:
let peak_position f =
  let xs = Mat.linspace (-5.) 5. 10000 in
  let p = Mat.map f xs in
  let _, id = Mat.max_i p in
  Mat.get xs id.(0) id.(1)

In [ ]:
let _ =
  let sigmas = Mat.linspace 0.1 2. 100 in
  let prior = gaussian ~mu:0. ~sigma:1. in
  let likelihood sigma = gaussian ~mu:1. ~sigma in
  let shifts =
    Mat.map (fun sigma -> 1. -. peak_position (prod prior (likelihood sigma))) sigmas
  in
  let fig (module P : Plot) =
    P.plot
      (L [ sigmas; shifts ])
      [ barebone
      ; borders [ `bottom; `left ]
      ; xlabel "likelihood spread"
      ; ylabel "posterior shift towards prior"
      ; xtics (`regular [ 0.; 0.5 ])
      ; ytics (`regular [ 0.; 0.2 ])
      ]
  in
  Juplot.draw ~size:(300, 200) fig